# Start code

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [12]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [13]:

train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)


# Creat model

In [14]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

In [15]:
model = model.to(device)

# Train model

In [16]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [18]:
# just to double check cuda is working
print(device.type)

epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

cuda
Epoch 0, iter 0, loss: 13926.45703125
Epoch 0, iter 10, loss: 13301.0849609375
Epoch 0, iter 20, loss: 13396.025390625
Epoch 0, iter 30, loss: 11770.970703125
Epoch 0, iter 40, loss: 12690.1357421875
Epoch 0, iter 50, loss: 11626.6630859375
Epoch 0, iter 60, loss: 11727.3642578125
Epoch 0, iter 70, loss: 10976.8779296875
Epoch 0, iter 80, loss: 11113.6630859375
Epoch 0, iter 90, loss: 10449.064453125
Epoch 0, iter 100, loss: 9744.17578125
Epoch 0, iter 110, loss: 9525.248046875
Epoch 0, iter 120, loss: 9352.3203125
Epoch 0, iter 130, loss: 8581.033203125
Epoch 0, iter 140, loss: 9166.91015625
Epoch 0, iter 150, loss: 7955.07177734375
Epoch 0, iter 160, loss: 7773.75830078125
Epoch 0, iter 170, loss: 7538.74365234375
Epoch 0, iter 180, loss: 7194.09521484375
Epoch 0, iter 190, loss: 7638.19140625
Epoch 0, iter 200, loss: 6936.47998046875
Epoch 0, iter 210, loss: 6730.21923828125
Epoch 0, iter 220, loss: 6130.73583984375
Epoch 0, iter 230, loss: 7605.9765625
Epoch 0, iter 240, loss:

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")